### Data Preparation for model

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### Load the 2013 data

In [32]:
data  = pd.read_csv("/home/smita/MP/train_all_2013.csv")
del data['year']
del data['Unnamed: 0']
print("Initially, we have", data.shape[0], "and", data.shape[1],"variables in our data including the response.")

Initially, we have 11186881 and 24 variables in our data including the response.


We create our downsampled data, we first spilt the data by booking and clicks event. Rationale behind this is our data is highyly
imbalanced in terms of booking data. We only 10 % booking data.


In [33]:
data1 = data[data.is_booking==1] # separate booking data
print("size of the booking data", data1.shape)
unique_users = data1.user_id.unique()
print("We have data for", len(unique_users), "users who booked the hotels")

size of the booking data (1024389, 24)
We have data for 387825 users who booked the hotels


In [34]:
data0 = data[data.is_booking==0] # separate click data
print("size of the click data", data0.shape)
unique_users = data0.user_id.unique()
print("We have data for", len(unique_users), "users who only cliked the hotels")

size of the click data (10162492, 24)
We have data for 634903 users who only cliked the hotels


#### This block of code randomly select booking data for 1,93,912 unique users

In [20]:
import random
random.seed(8)
unique_users = data1.user_id.unique()
print("We have data for", len(unique_users), "users")
sel_user_ids = [unique_users[i] for i in sorted(random.sample(range(len(unique_users)), 193912)) ]
train1 = data1[data1.user_id.isin(sel_user_ids)]
print("new sample contains", train1.shape[0], "rows and", train1.shape[1], "columns")

We have data for 387825 users
new sample contains 512950 rows and 26 columns


### This block of code randomly select click data for 60000 unique users

In [23]:
import random
random.seed(8)
unique_users = data0.user_id.unique()
print("We have data for", len(unique_users), "users")
sel_user_ids = [unique_users[i] for i in sorted(random.sample(range(len(unique_users)), 60000)) ]
train2 = data0[data0.user_id.isin(sel_user_ids)]
print("new sample contains", train2.shape[0], "rows and", train2.shape[1], "columns")

We have data for 634903 users
new sample contains 958461 rows and 26 columns


In [36]:
# Combine click and booking data
train = train1.append(train2)
print("training set constitute", train.shape[0], "rows")

training set constitute 1471411 rows


### Test Set up

In [27]:
data  = pd.read_csv("/home/smita/MP/train_all_2014.csv")

In [30]:
# This block of code randomly select 2014 booking data for 50000 unique users in 2013
import random
random.seed(12)
unique_users = train.user_id.unique()
print("We have", len(unique_users), "unique users in the training set")
sel_user_ids = [unique_users[i] for i in sorted(random.sample(range(len(unique_users)), 50000)) ]
test = data[data.user_id.isin(sel_user_ids)]
test = test[test.is_booking ==1]
print("new test contains", test.shape[0], "rows and", test.shape[1], "columns")

We have 235750 unique users in the training set
new test contains 123506 rows and 26 columns


In [37]:
test.columns.values

array(['Unnamed: 0', 'date_time', 'site_name', 'posa_continent',
       'user_location_country', 'user_location_region',
       'user_location_city', 'orig_destination_distance', 'user_id',
       'is_mobile', 'is_package', 'channel', 'srch_ci', 'srch_co',
       'srch_adults_cnt', 'srch_children_cnt', 'srch_rm_cnt',
       'srch_destination_id', 'srch_destination_type_id', 'is_booking',
       'cnt', 'hotel_continent', 'hotel_country', 'hotel_market',
       'hotel_cluster', 'year'], dtype=object)

In [39]:
test.shape

(123506, 24)

In [40]:
## save this state for later use
#train.to_csv("/home/smita/Final/Sampledtrain2013.csv")
#test.to_csv("/home/smita/Final/test2014.csv")

Next, we use '~/Final/data_generation_script1.R' to create feature using discretization process.

In [44]:
train = pd.read_csv("~/Final/Sampledtrain2013Stage1.csv")
test = pd.read_csv("~/Final/test2014Stage1.csv")

In [64]:
newVar = [26, 27, 28, 29, 30, 32, 33, 34, 36, 37, 38]
train.iloc[1:3, newVar]

,hotel_cluster,year,month,day,hour,type_of_day,season,is_alone,dest_feature_pc1,dest_feature_pc2,dest_feature_pc3
1,30,2013,3,31,22,0,1,1.0,1.153878,0.303059,0.92731
2,20,2013,4,3,0,1,1,0.0,1.153878,0.303059,0.92731


In [65]:
test.iloc[1:3, newVar]

,year,month,day,hour,part_of_day,season,duration_of_stay,is_alone,dest_feature_pc1,dest_feature_pc2,dest_feature_pc3
1,2014,8,13,13,2.0,2,4,1.0,-3.727968,-0.584871,0.403375
2,2014,12,28,7,1.0,4,2,1.0,-3.727968,-0.584871,0.403375


In [66]:
del train
del test 
gc.collect()

271

Next, we create features using books clicks using script "~/Final/data_preprationBookclicks.R"

In [87]:
train = pd.read_csv("~/Final/sampleTrain2013stage2.csv")
test = pd.read_csv("~/Final/test2014stage2.csv")

In [91]:
#del indice column
selectCols = ['user_location_country', 'hotel_cluster',
       'user_location_region', 'user_location_city', 'hotel_country',
       'hotel_market', 'srch_destination_id',
       'date_time', 'site_name', 'posa_continent',
       'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
       'channel', 'srch_ci', 'srch_co', 'srch_adults_cnt',
       'srch_children_cnt', 'srch_rm_cnt', 'srch_destination_type_id',
       'is_booking', 'cnt', 'hotel_continent', 'year', 'month', 'day',
       'hour', 'part_of_day', 'type_of_day', 'season', 'is_alone',
       'dest_feature_pc1', 'dest_feature_pc2', 'dest_feature_pc3',
       'clickRate', 'bookRate', 'clickMRate', 'bookMRate', 'clickHCRate',
       'bookHCRate', 'clickURate', 'bookURate', 'clickUCRRate',
       'bookUCRRate', 'clickUCRate', 'bookUCRate']

In [95]:
train = train[selectCols]
test = test[selectCols]

In [97]:
train.shape

(1471411, 47)

In [98]:
test.shape

(123506, 47)

##### Stage 3

In [2]:
train = pd.read_csv("~/cleantrain2013Stage3.csv")
test = pd.read_csv("~/cleantest2014Stage3.csv")

In [3]:
del train['Unnamed: 0']
del test['Unnamed: 0']

In [4]:
train.shape

(1471411, 47)

In [5]:
test.shape

(123506, 47)

In [14]:
unique_users = train.user_id.unique()
print("We have", len(unique_users), "unique users in the training set")

('We have', 235750, 'unique users in the training set')


### Add new features using books click weightage

In [6]:

"Create book and click Scores:"
    
#Get the book/click rate by hotel attributes and  month. This is in a hope we willl capture 
#which hotels popular in a particular month""

columns = ['month', 'hotel_continent','hotel_country','is_booking']
grps = ['month', 'hotel_continent','hotel_country']
data = train[columns]
l = data.groupby(grps)['is_booking'].agg(['sum', 'count'])
l['popScoreHmc'] = l['sum']+0.25*0.75 + l['count']*0.40  # We give some random weight. However, we give booking always a high weight
del l['sum']
del l['count']
l = l.add_suffix('_Count').reset_index()

train = train.merge(l, how='left', left_on=grps, right_on=grps)
test = test.merge(l, how='left', left_on=grps, right_on=grps)


# Get the clicks and booking score bu user attributes
columns = ['user_location_country', 'user_location_region','user_location_city','is_booking']
grps = ['user_location_country', 'user_location_region','user_location_city']
data = train[columns]
l = data.groupby(grps)['is_booking'].agg(['sum', 'count'])
l['popScoreU'] = l['sum']+0.25*0.75 + l['count']*0.40
del l['sum']
del l['count']
l = l.add_suffix('_Count').reset_index()

train = train.merge(l, how='left', left_on=grps, right_on=grps)
test = test.merge(l, how='left', left_on=grps, right_on=grps)


# Get the clicks and booking site and cotinent like .com 

columns = ['posa_continent', 'site_name','hotel_market','is_booking']
grps = ['posa_continent', 'site_name','hotel_market']
data = train[columns]
l = data.groupby(grps)['is_booking'].agg(['sum', 'count'])
l['popScoreSite'] = l['sum']+0.15*0.75 + l['count']*0.45
del l['sum']
del l['count']
l = l.add_suffix('_Count').reset_index()
train = train.merge(l, how='left', left_on=grps, right_on=grps)
test = test.merge(l, how='left', left_on=grps, right_on=grps)


# Get the scores by search dest and type

columns = ['srch_destination_id', 'srch_destination_type_id','is_booking']
grps = ['srch_destination_id', 'srch_destination_type_id']
data = train[columns]
l = data.groupby(grps)['is_booking'].agg(['sum', 'count'])
l['popScoreDest'] = l['sum']+0.1*0.50 + l['count']*0.30
del l['sum']
del l['count']
l = l.add_suffix('_Count').reset_index()

train = train.merge(l, how='left', left_on=grps, right_on=grps)
test = test.merge(l, how='left', left_on=grps, right_on=grps)

# Get the clicks and booking based on destination and user city

columns = ['srch_destination_id', 'user_location_city','is_booking']
grps = ['srch_destination_id', 'user_location_city']
data = train[columns]
l = data.groupby(grps)['is_booking'].agg(['sum', 'count'])
l['popScoreCityDest'] = l['sum']+0.2*0.70 + l['count']*0.40
del l['sum']
del l['count']
l = l.add_suffix('_Count').reset_index()

train = train.merge(l, how='left', left_on=grps, right_on=grps)
test = test.merge(l, how='left', left_on=grps, right_on=grps)

# Get the clicks and booking based on destination and user city

columns = ['user_location_region', 'user_location_country', 'hotel_country','is_booking']
grps =  ['user_location_region', 'user_location_country', 'hotel_country']
data = train[columns]
l = data.groupby(grps)['is_booking'].agg(['sum', 'count'])
l['popScoreCityH'] = l['sum']+0.3*0.65 + l['count']*0.50
del l['sum']
del l['count']
l = l.add_suffix('_Count').reset_index()
train = train.merge(l, how='left', left_on=grps, right_on=grps)
test = test.merge(l, how='left', left_on=grps, right_on=grps)



In [7]:
train.shape

(1471411, 53)

In [8]:
test.shape

(123506, 53)

In [10]:
train.columns.values

array(['user_location_country', 'hotel_cluster', 'user_location_region',
       'user_location_city', 'hotel_country', 'hotel_market',
       'srch_destination_id', 'date_time', 'site_name', 'posa_continent',
       'orig_destination_distance', 'user_id', 'is_mobile', 'is_package',
       'channel', 'srch_ci', 'srch_co', 'srch_adults_cnt',
       'srch_children_cnt', 'srch_rm_cnt', 'srch_destination_type_id',
       'is_booking', 'cnt', 'hotel_continent', 'year', 'month', 'day',
       'hour', 'part_of_day', 'type_of_day', 'season', 'is_alone',
       'dest_feature_pc1', 'dest_feature_pc2', 'dest_feature_pc3',
       'clickRate', 'bookRate', 'clickMRate', 'bookMRate', 'clickHCRate',
       'bookHCRate', 'clickURate', 'bookURate', 'clickUCRRate',
       'bookUCRRate', 'clickUCRate', 'bookUCRate', 'popScoreHmc_Count',
       'popScoreU_Count', 'popScoreSite_Count', 'popScoreDest_Count',
       'popScoreCityDest_Count', 'popScoreCityH_Count'], dtype=object)

In [12]:
train.iloc[1:3, 47:]

,popScoreHmc_Count,popScoreU_Count,popScoreSite_Count,popScoreDest_Count,popScoreCityDest_Count,popScoreCityH_Count
1,7281.5875,884.1875,1.5625,529.95,1.54,10.695
2,52363.9875,3.5875,12.4125,6686.35,1.34,164.695


In [13]:
test.iloc[1:3, 47:]

,popScoreHmc_Count,popScoreU_Count,popScoreSite_Count,popScoreDest_Count,popScoreCityDest_Count,popScoreCityH_Count
1,839.7875,295.5875,86.5625,5307.35,1.54,34.195
2,693.9875,901.7875,4020.7625,835.25,0.94,34.195


In [14]:
### Save this stage:
train.to_csv("~/cleantrain2013Stage4.csv")
test.to_csv("~/cleantest2014Stage4.csv")

cleantrain2013Stage4 and cleantest2014Stage4 contains features created discretization, PCA, historic book/click rates

### Data Cleaning

This script takes care of imputing the missing values.

In [6]:
train = pd.read_csv("~/cleantrain2013Stage4.csv")
test = pd.read_csv("~/cleantest2014Stage4.csv")

In [12]:
## duration_of_stay in both train and test set
train["srch_ci"] = pd.to_datetime(train["srch_ci"], format='%Y-%m-%d', errors="coerce")
train["srch_co"] = pd.to_datetime(train["srch_co"], format='%Y-%m-%d', errors="coerce")
train["duration_of_stay"] = (train["srch_co"]-train["srch_ci"]).astype('timedelta64[h]')
train["duration_of_stay"] = train["duration_of_stay"]/24

test["srch_ci"] = pd.to_datetime(test["srch_ci"], format='%Y-%m-%d', errors="coerce")
test["srch_co"] = pd.to_datetime(test["srch_co"], format='%Y-%m-%d', errors="coerce")
test["duration_of_stay"] = (test["srch_co"]-test["srch_ci"]).astype('timedelta64[h]')
test["duration_of_stay"] = test["duration_of_stay"]/24

In [13]:
missing_count = train.count(axis=0)
missing_count = train.count(axis=0)
missing = pd.DataFrame(missing_count)


In [10]:
missing.transpose().iloc[0:3, 1:10]

,user_location_country,hotel_cluster,user_location_region,user_location_city,hotel_country,hotel_market,srch_destination_id,date_time,site_name
0,1471411,1471411,1471411,1471411,1471411,1471411,1471411,1471411,1471411


In [14]:
missing.transpose().iloc[0:3, 11:20]

,orig_destination_distance,user_id,is_mobile,is_package,channel,srch_ci,srch_co,srch_adults_cnt,srch_children_cnt
0,947444,1471411,1471411,1471411,1471411,1467879,1467879,1471411,1471411


We have many missing values in the origin_destination_distance. We do not have any better way to impute these missing value
Therefore, we will simply impute it by -1

In [15]:
train['orig_destination_distance'].fillna(-1, inplace=True)
test['orig_destination_distance'].fillna(-1, inplace=True)

In [16]:
missing.transpose().iloc[0:3, 21:30]

,srch_destination_type_id,is_booking,cnt,hotel_continent,year,month,day,hour,part_of_day
0,1471411,1471411,1471411,1471411,1471411,1471411,1471411,1471411,1444348


In [19]:
## We impute part_of_day by 3 i.e. evening
train['part_of_day'].fillna(3, inplace=True)
test['part_of_day'].fillna(3, inplace=True)

In [20]:
missing.transpose().iloc[0:3, 31:40]

,season,is_alone,dest_feature_pc1,dest_feature_pc2,dest_feature_pc3,clickRate,bookRate,clickMRate,bookMRate
0,1471411,1468454,1467156,1467156,1467156,1460844,1460059,1471273,1471133


In [21]:
## We impute is_alone by 1
train['is_alone'].fillna(1, inplace=True)
test['is_alone'].fillna(1, inplace=True)

In [22]:
## We impute dest_feature_pc1 by -1
train['dest_feature_pc1'].fillna(-1, inplace=True)
test['dest_feature_pc1'].fillna(-1, inplace=True)

train['dest_feature_pc2'].fillna(-1, inplace=True)
test['dest_feature_pc2'].fillna(-1, inplace=True)

train['dest_feature_pc3'].fillna(-1, inplace=True)
test['dest_feature_pc3'].fillna(-1, inplace=True)


In [25]:
# Replace all the missing rates by 0.0
ratesVar = ['clickRate', 'bookRate', 'clickMRate',
       'bookMRate', 'clickHCRate', 'bookHCRate', 'clickURate', 'bookURate',
       'clickUCRRate', 'bookUCRRate', 'clickUCRate', 'bookUCRate']
for col in ratesVar:
    train[col].fillna(0.0, inplace=True)
    test[col].fillna(0.0, inplace=True)


In [41]:
# Replace all the missing rates by 0.0
popScore = ['popScoreHmc_Count', 'popScoreU_Count', 'popScoreSite_Count',
           'popScoreDest_Count', 'popScoreCityDest_Count',
              'popScoreCityH_Count']
for col in popScore:
    train[col].fillna(0.0, inplace=True)
    test[col].fillna(0.0, inplace=True)

In [29]:
missing.transpose().iloc[0:3, 48:]


,popScoreHmc_Count,popScoreU_Count,popScoreSite_Count,popScoreDest_Count,popScoreCityDest_Count,popScoreCityH_Count,duration_of_stay
0,1471411,1471411,1471411,1471411,1471411,1471411,1467879


In [30]:
# duration of stay by 1
train['duration_of_stay'].fillna(1, inplace=True)
test['duration_of_stay'].fillna(1, inplace=True)

Verify Missing Data Again:

In [42]:
missing_count = train.count(axis=0)
missing_count = train.count(axis=0)
missing = pd.DataFrame(missing_count)


In [43]:
missing_count

Unnamed: 0                   1471411
user_location_country        1471411
hotel_cluster                1471411
user_location_region         1471411
user_location_city           1471411
hotel_country                1471411
hotel_market                 1471411
srch_destination_id          1471411
date_time                    1471411
site_name                    1471411
posa_continent               1471411
orig_destination_distance    1471411
user_id                      1471411
is_mobile                    1471411
is_package                   1471411
channel                      1471411
srch_ci                      1467879
srch_co                      1467879
srch_adults_cnt              1471411
srch_children_cnt            1471411
srch_rm_cnt                  1471411
srch_destination_type_id     1471411
is_booking                   1471411
cnt                          1471411
hotel_continent              1471411
year                         1471411
month                        1471411
d

In [44]:
missing_count = test.count(axis=0)
missing_count = test.count(axis=0)
missing = pd.DataFrame(missing_count)
missing

,0
Unnamed: 0,123506
user_location_country,123506
hotel_cluster,123506
user_location_region,123506
user_location_city,123506
hotel_country,123506
hotel_market,123506
srch_destination_id,123506
date_time,123506
site_name,123506


In [45]:
train.shape

(1471411, 55)

In [46]:
test.shape

(123506, 55)

We again downsample data as we have to use evaluate performance of multiple algorithm:
    

In [49]:
import random
random.seed(24)
unique_users = train.user_id.unique()
print("We have data for", len(unique_users), "users")
sel_user_ids = [unique_users[i] for i in sorted(random.sample(range(len(unique_users)), 80000)) ]
train_sample = train[train.user_id.isin(sel_user_ids)]
print("new sample contains", train_sample.shape[0], "rows and", train_sample.shape[1], "columns")

('We have data for', 235750, 'users')
('new sample contains', 495745, 'rows and', 55, 'columns')


In [50]:
#Select data for selected user from test set
test_sample = test[test.user_id.isin(sel_user_ids)]

In [51]:
test_sample.shape

(41597, 55)

In [52]:
len(train.hotel_cluster.unique())

100

In [53]:
data = train_sample[train_sample.is_booking==1]
data.shape

(173973, 55)

In [54]:
data = train_sample[train_sample.is_booking==0]
data.shape

(321772, 55)

In [ ]:
We have a good balance of clicks and booking events in our final set.

In [55]:
train_sample.to_csv("~/FinalTrainingSet.csv")
test_sample.to_csv("~/FinalTestSet.csv")